In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed
tf.keras.backend.clear_session()
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from keras.regularizers import l2

In [ ]:
# reading the dataframe from disk

data = pd.read_csv('/content/drive/MyDrive/seq2seq/data.csv')
data.head()

,corrupted_text,english_text
0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


## Data Augmentation

In [ ]:
!pip install nlpaug

     |████████████████████████████████| 405 kB 5.1 MB/s 


In [ ]:
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
from nlpaug.util.file.download import DownloadUtil

In [ ]:
model_dir = '/content/'
DownloadUtil.download_glove('glove.6B', '/content/')

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# reference: https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb

aug_key_char = nac.KeyboardAug()

# aug_insert = naw.WordEmbsAug(
#     model_type='glove', model_path=model_dir+'glove.6B.300d.txt',
#     action="insert")

aug_substitute = naw.WordEmbsAug(
    model_type='glove', model_path=model_dir+'glove.6B.300d.txt',
    action="substitute")

aug_synonym = naw.SynonymAug(aug_src='wordnet')

In [ ]:
aug_char_dist = list()
# aug_text_insert = list()
aug_text_substitute = list()
aug_text_synonym = list()

for text in tqdm(data.english_text.values):
    aug_char_dist.append(aug_key_dist.augment(text))
    # aug_text_insert.append(aug_insert.augment(text))
    aug_text_substitute.append(aug_substitute.augment(text))
    aug_text_synonym.append(aug_synonym.augment(text))

100%|██████████| 2000/2000 [14:59<00:00,  2.22it/s]


In [ ]:
char_df = pd.DataFrame(list(zip(aug_char_dist, data.english_text.values)), columns=['corrupted_text', 'english_text'])
text_substitute_df = pd.DataFrame(list(zip(aug_text_substitute, data.english_text.values)), columns=['corrupted_text', 'english_text'])
synonym_df = pd.DataFrame(list(zip(aug_text_synonym, data.english_text.values)), columns=['corrupted_text', 'english_text'])

df = pd.concat([text_substitute_df, synonym_df])
df.head()

,corrupted_text,english_text
0,Do thing want me to reserve re-elected for you...,Do you want me to reserve seat for you or not?\n
1,Yeap. You higher? We announced some Durian pas...,Yeap. You reaching? We ordered some Durian pas...
2,They become but expensive although. Mine appea...,They become more expensive already. Mine is li...
3,I ' j Thai. What even can do?,I'm Thai. What do you do?\n
4,Hi! How did your tuesday go? Haven ' shorts he...,Hi! How did your week go? Haven't heard from y...


In [ ]:
data_df = pd.concat([data, df])
data_df.head()

,corrupted_text,english_text
0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [ ]:
# saving the file to disk
# data_df.to_csv('/content/drive/MyDrive/seq2seq/data_df.csv', index=False)

## Preprocessing

In [7]:
data_df = pd.read_csv('/content/drive/MyDrive/seq2seq/data_df.csv')

In [8]:
# reference: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [9]:
def preprocess(text):
    """
        Function to clean the strings containing special characters and converts them to lowercase characters.

        input: string
        output: string which contains number and lower character.
    """

    # convert the string to lowercase
    text = text.lower()
    # decontraction - expanding the words like : i'll -> i will, he'd -> he would
    text = decontracted(text)
    text = re.sub('[^A-Za-z0-9]',' ',text)
    text = re.sub('\s_\s', ' ', text)   #  replace strings like  ' _ ' with ' ' (string with a space)
    text = re.sub('\s+', ' ', text).strip()  # replace more than one_space_character to single_space_character

    return text

In [10]:
# Create new `pandas` methods which use `tqdm` progress

tqdm.pandas()
data_df['corrupted_text'] = data_df['corrupted_text'].progress_apply(preprocess)
data_df['english_text']   = data_df['english_text'].progress_apply(preprocess)

100%|██████████| 6000/6000 [00:00<00:00, 37080.00it/s]


In [11]:
data_df.head()

,corrupted_text,english_text
0,u wan me to chop seat 4 u nt,do you want me to reserve seat for you or not
1,yup u reaching we order some durian pastry alr...,yeap you reaching we ordered some durian pastr...
2,they become more ex oredi mine is like 25 so h...,they become more expensive already mine is lik...
3,i am thai what do u do,i am thai what do you do
4,hi how did your week go haven heard from you f...,hi how did your week go have not heard from yo...


In [12]:
# shuffling the dataframe 
data_df = data_df.sample(frac=1).reset_index(drop=True)

## Data Tokenization

In [13]:
english_text_inp = '<start> ' + data_df['english_text'].astype(str)
english_text_out = data_df['english_text'].astype(str) + ' <end>'

In [14]:
data_df['english_inp'] = english_text_inp
data_df['english_out'] = english_text_out

In [15]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_df, test_size=0.1)
train, test = train_test_split(train, test_size=0.1)

In [16]:
# we are doing the below step to add '<end>' to the tokenizer dictionary

train.iloc[0]['english_inp'] = train.iloc[0]['english_inp'] + ' <end>'

In [17]:
tknizer_corr = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_corr.fit_on_texts(train['corrupted_text'].values)

tknizer_eng = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_eng.fit_on_texts(train['english_inp'].values)

In [18]:
vocab_size_eng=len(tknizer_eng.word_index.keys())
print(vocab_size_eng)
vocab_size_corr=len(tknizer_corr.word_index.keys())
print(vocab_size_corr)

2988
7590


In [19]:
class Dataset:
    def __init__(self, data, tknizer_corr, tknizer_eng, max_len):
        self.encoder_inps = data['corrupted_text'].values
        self.decoder_inps = data['english_inp'].values
        self.decoder_outs = data['english_out'].values
        self.tknizer_eng = tknizer_eng
        self.tknizer_corr = tknizer_corr
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_corr.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [20]:
train_dataset = Dataset(train, tknizer_corr, tknizer_eng, 50)
validation_dataset  = Dataset(validation, tknizer_corr, tknizer_eng, 50)

train_dataloader = Dataloder(train_dataset, batch_size=100)
validation_dataloader = Dataloder(validation_dataset, batch_size=100)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(100, 50) (100, 50) (100, 50)


## Encoder Decoder model


In [21]:
# Encoder setup
encoder_input = Input(shape=(None, ), name='encoder_input')
embedding_encoder = Embedding(input_dim=vocab_size_corr + 1 , output_dim=50 , mask_zero=True , name='embedding_layer_encoder')(encoder_input)
lstm_encoder, state_h, state_c = LSTM(124  , return_state=True, return_sequences=True, name='Encoder_LSTM')(embedding_encoder)
encoder_states = [state_h, state_c]

# Decoder setup
decoder_input = Input(shape=(None, ), name='decoder_input')
embedding_output = Embedding(input_dim=vocab_size_eng + 1 , output_dim=100 ,mask_zero= True, name='embedding_layer_decoder')
decoder_embedding = embedding_output(decoder_input)

lstm_decoder = LSTM(124 , return_state=True, return_sequences=True, name='Decoder_LSTM')
decoder_output, _, _ = lstm_decoder(decoder_embedding, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(vocab_size_eng + 1 , activation='softmax'))
dense_layer = decoder_dense(decoder_output)

model = Model([encoder_input, decoder_input], dense_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_layer_encoder (Embedd (None, None, 50)     379550      encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_layer_decoder (Embedd (None, None, 100)    298900      decoder_input[0][0]              
______________________________________________________________________________________________

In [22]:
import datetime
filepath = '/content/drive/MyDrive/seq2seq/checkpoint/model_1/weights-{epoch:02d}-{val_loss:.4f}.hdf5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=0, save_best_only=True, mode="auto")

# log_dir = '/content/drive/MyDrive/seq2seq/logs/model_1/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir="/content/drive/MyDrive/seq2seq/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.compat.v1.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_grads =True)

In [23]:
train.shape, validation.shape, test.shape

((4860, 4), (600, 4), (540, 4))

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

train_steps=train.shape[0]//128
valid_steps=validation.shape[0]//128

model_history1=model.fit_generator(train_dataloader,steps_per_epoch=train_steps,epochs=100, validation_data=validation_dataloader, validation_steps=valid_steps,
                                   callbacks=[
                                              model_checkpoint, 
                                              tensorboard_callback
                                              ]
                                   )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
37/37 [==============================] - 20s 195ms/step - loss: 2.2341 - accuracy: 0.0663 - val_loss: 1.8864 - val_accuracy: 0.0659
Epoch 2/100
37/37 [==============================] - 3s 86ms/step - loss: 1.8264 - accuracy: 0.0800 - val_loss: 1.8605 - val_accuracy: 0.0823
Epoch 3/100
37/37 [==============================] - 3s 86ms/step - loss: 1.7942 - accuracy: 0.0984 - val_loss: 1.8377 - val_accuracy: 0.1037
Epoch 4/100
37/37 [==============================] - 3s 80ms/step - loss: 1.7924 - accuracy: 0.1068 - val_loss: 1.8104 - val_accuracy: 0.1018
Epoch 5/100
37/37 [==============================] - 3s 78ms/step - loss: 1.7457 - accuracy: 0.1092 - val_loss: 1.7806 - val_accuracy: 0.1045
Epoch 6/100
37/37 [==============================] - 3s 79ms/step - loss: 1.7112 - accuracy: 0.1105 - val_loss: 1.7632 - val_accuracy: 0.1099
Epoch 7/100
37/37 [==============================] - 3s 78ms/step - loss: 1.7182 - accuracy: 0.1137 - val_loss: 1.7513 - val_accuracy: 0.1143
Epoc

## Inference

In [25]:
encoder_model = Model(encoder_input, encoder_states)

decoder_state_h = Input(shape=(None,), name='decoder_state_h')
decoder_state_c = Input(shape=(None,), name='decoder_state_c')
decoder_states = [decoder_state_h, decoder_state_c]

decoder_embed = embedding_output(decoder_input)
decoder_lstm, state_h_, state_c_ = lstm_decoder(decoder_embed, initial_state=decoder_states)
decoder_state_out = [state_h_, state_c_]
decoder_out = model.layers[6](decoder_lstm)

In [26]:
decoder_model = Model([decoder_input] + decoder_states, [decoder_out] + decoder_state_out)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_layer_decoder (Embedd (None, None, 100)    298900      decoder_input[0][0]              
__________________________________________________________________________________________________
decoder_state_h (InputLayer)    [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_state_c (InputLayer)    [(None, None)]       0                                            
____________________________________________________________________________________________

In [27]:
def inference(sequence):
    inp_seq = tknizer_corr.texts_to_sequences([sequence])
    inp_seq = pad_sequences(inp_seq, maxlen=50, dtype='int32', padding='post')

    inp_seq = inp_seq.reshape(-1,50)
    encoder_states = encoder_model.predict(inp_seq)

    target = tf.constant([[tknizer_eng.word_index['<start>']]])
    pred = []
    for i in range(20):
        output, state_h, state_c = decoder_model([target]+encoder_states)
        encoder_states = [state_h, state_c]
        target = np.reshape(np.argmax(output), (1, 1))
        pred.append(tknizer_eng.index_word[target[0][0]])
        if(pred[-1]=='<end>'):
            break
    translated_sentence = ' '.join(pred)

    return translated_sentence

In [32]:
t = test.corrupted_text.values[0:10]
l = test.english_text.values[0:10]
s = list()
for i in t:
    k = inference(i)
    s.append(k)

In [33]:
for i in range(10):
    print('actual    :-', l[i])
    print('predicted :-', s[i])
    print('-' * 20)
    print()

actual    :- yup hehe you like then good hehe
predicted :- yes it is it to buy newspaper <end>
--------------------

actual    :- i think i do not want the glasses seldom use it anyway save some money hee
predicted :- i will be late have not ask her to wait for rebecca she is not <end>
--------------------

actual    :- i think the chinese tester is better
predicted :- i am going to claim vat <end>
--------------------

actual    :- no i am going to the wrks with j what about you
predicted :- no i am not going to school then can you be with <end>
--------------------

actual    :- what time will you end then we are at og
predicted :- what time will you be at home <end>
--------------------

actual    :- relax it is easy i am fine thanks my friends are asking me out but i do not know what to do now so boring
predicted :- i am in imperialmusic listening to the weirdest track ever by leafcutter john sounds like insects being molested and someone
--------------------

actual    :- hi kote 